In [ ]:
#
#    НА ОШИБКУ НЕ ОБРАЩАЙТЕ ВНИМАНИЕ
#
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 
import operator
from matplotlib import pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
data = pd.read_csv('../input/wages/wages.csv')
data = data.drop(data.columns[[0,1, 3, 5]], axis=1)
data.drop(data.index[100:], inplace=True)
data.loc[data.sex != 'female', 'female'] = 0
data.loc[data.sex == 'female', 'female'] = 1
data['sex'] = data['female']
data = data.drop(data.columns[[2]], axis=1)
print(data)



n = len(data)
mm = 2
m = 2
k = 2
MAX_ITER = 100


def printArr(arr):
    for i in range(n):
        for j in range(mm):
            print(arr[i][j], '  ', end = ' ')  ### Функция вывода матрицы
        print()
        
def subtract_vectors(v, w):
    return [vi - wi for vi, wi in zip(v, w)]  ## Функция вычитания векторов

def initialize():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]    ## Генерируется матрица u(i, j)
        summation = 1/sum(random_num_list)
        temp_list = [summation*x for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat



def calculateClusterCenter(membership_mat):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]                ####### Функция считает c
        denominator = sum(xraised) 
        temp_num = list()
        for i in range(n):
            data_point = list(data.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, list(zip(*temp_num)))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers



def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(data.iloc[i])
        distances = [np.linalg.norm(subtract_vectors(x, cluster_centers[j])) for j in range(k)] ## Обновили u
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat


def getClusters(membership_mat):
    cluster_labels = list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i])) ## Выбирает кластер для наблюдения
        cluster_labels.append(idx)
    return cluster_labels


def fuzzyCMeansClustering():
    # Membership Matrix
    membership_mat = initialize()
    curr = 0
    while curr <= MAX_ITER:
        cluster_centers = calculateClusterCenter(membership_mat)
        membership_mat2 = updateMembershipValue(membership_mat, cluster_centers)    # Сравнивает по эпсил. 
        cluster_labels = getClusters(membership_mat)
        curr += 1
    print(cluster_labels)
    printArr(membership_mat)
    return membership_mat, membership_mat2, cluster_labels, cluster_centers

membership_mat,membership_mat2, fcm_labels, fcm_centers = fuzzyCMeansClustering()

fcm_labels = np.array(fcm_labels)
fcm_centers = np.array(fcm_centers)                       # Массивы в NumPy для рисунков
membership_mat2 = np.array(membership_mat2)

membership_mat = np.array(membership_mat)


x_values = [0.0, 0.2, 0.4, 0.6, 0.8, 1]
f, axes = plt.subplots(1, 2, figsize=(11,5))
axes[0].scatter(membership_mat[:,0], membership_mat[:,1], alpha=.1)
axes[1].scatter(membership_mat2[:,0], membership_mat2[:,1], c=fcm_labels, alpha=.1)
xticks(x_values)
axes[1].scatter(fcm_centers[:,0], fcm_centers[:,1], marker="+", s=500, c='w')
xticks(x_values)

plt.show()




In [ ]:
pip install fuzzy-c-means

In [ ]:
pip install fuzzy-c-means

In [ ]:
#
#     ЧИТЕРСКИЙ МЕТОД, НЕ ФАКТ, ЧТО ПРАВИЛЬНЫЙ
#
%matplotlib inline
import numpy as np
from fcmeans import FCM
from matplotlib import pyplot as plt
n = len(data)
mm = 3
m = 2
k = 3
MAX_ITER = 100
n_samples = 3000
data = pd.read_csv('../input/wages/wages.csv')
data = data.drop(data.columns[[1, 3, 5]], axis=1)
data.drop(data.index[100:], inplace=True)
data.loc[data.sex != 'female', 'female'] = 0
data.loc[data.sex == 'female', 'female'] = 1
data['sex'] = data['female']
data = data.drop(data.columns[[3]], axis=1)

def initialize():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]
        summation = 1/sum(random_num_list)
        temp_list = [summation*x for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat

def calculateClusterCenter(membership_mat):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]
        denominator = sum(xraised)
        temp_num = list()
        for i in range(n):
            data_point = list(data.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, list(zip(*temp_num)))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers

mat = initialize()
mat2 = calculateClusterCenter(mat)

def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(data.iloc[i])
        distances = [np.linalg.norm(subtract_vectors(x, cluster_centers[j])) for j in range(k)]
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat
mat3 = updateMembershipValue(mat, mat2)

mat3 = np.array(mat3)
mat = np.array(mat)



fcm = FCM(n_clusters=3)
fcm.fit(mat3)

# outputs
fcm_centers = fcm.centers
fcm_labels = fcm.predict(mat3)

# plot result
f, axes = plt.subplots(1, 2, figsize=(11,5))
axes[0].scatter(mat3[:,0], mat3[:,1], alpha=.1)
axes[1].scatter(mat3[:,0], mat3[:,1], c=fcm_labels, alpha=.1)
axes[1].scatter(fcm_centers[:,0], fcm_centers[:,1], marker="+", s=500, c='w')
plt.show()

In [ ]:
#
#              НЕ ОБРАЩАЙТЕ ВНИМАНИЕ
#
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 
import operator
from matplotlib import pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
data = pd.read_csv('../input/wages/wages.csv')

data = data.drop(data.columns[[1, 3, 5]], axis=1)
data.drop(data.index[100:], inplace=True)
data.loc[data.sex != 'female', 'female'] = 0
data.loc[data.sex == 'female', 'female'] = 1
data['sex'] = data['female']
data = data.drop(data.columns[[3]], axis=1)



n = len(data)
mm = 3
m = 2
k = 3
MAX_ITER = 100


def printArr(arr):
    for i in range(n):
        for j in range(mm):
            print(arr[i][j], '  ', end = ' ')
        print()
        
def subtract_vectors(v, w):
    return [vi - wi for vi, wi in zip(v, w)]

def initialize():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]
        summation = 1/sum(random_num_list)
        temp_list = [summation*x for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat

mat = initialize()

def calculateClusterCenter(membership_mat):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]
        denominator = sum(xraised)
        temp_num = list()
        for i in range(n):
            data_point = list(data.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, list(zip(*temp_num)))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers

mat2 = calculateClusterCenter(mat)

def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(data.iloc[i])
        distances = [np.linalg.norm(subtract_vectors(x, cluster_centers[j])) for j in range(k)]
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat


def getClusters(membership_mat):
    cluster_labels = list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
        cluster_labels.append(idx)
    return cluster_labels


def fuzzyCMeansClustering():
    # Membership Matrix
    membership_mat = initialize()
    curr = 0
    while curr <= MAX_ITER:
        cluster_centers = calculateClusterCenter(membership_mat)
        membership_mat = updateMembershipValue(membership_mat, cluster_centers)
        cluster_labels = getClusters(membership_mat)
        curr += 1
    printArr(membership_mat)
    return cluster_labels, cluster_centers

fcm_labels, fcm_centers = fuzzyCMeansClustering()

fcm_labels = np.array(fcm_labels)
print(fcm_labels)
fcm_centers = np.array(fcm_centers)
print(fcm_centers)
mat = np.array(mat)
print(mat)
printArr(mat)

f, axes = plt.subplots(1, 2, figsize=(11,5))
axes[0].scatter(mat[:,0], mat[:,1], alpha=.1)
axes[1].scatter(mat[:,0], mat[:,1], c=fcm_labels, alpha=.1)
axes[1].scatter(fcm_centers[:,0], fcm_centers[:,1], marker="+", s=500, c='w')

plt.show()


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split 
import pandas as pd
data = pd.read_csv('../input/wages/wages.csv')
data = data.drop(data.columns[[2, 3]], axis=1)
data.drop(data.index[300:], inplace=True)
data = data.drop(['ed'],axis=1) 


#Separating the dependent and independent data variables into two data frames.
X = data.drop(['earn'],axis=1) 
Y = data['earn']

 
# Splitting the dataset into 80% training data and 20% testing data.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.20, random_state=0)
print(X_train, 'k')
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

from sklearn.linear_model import LinearRegression
linear_model = LinearRegression().fit(X_train , Y_train)
 
#Predictions on Testing data
LR_Test_predict = linear_model.predict(X_test) 

 
# Using MAPE error metrics to check for the error rate and accuracy level
LR_MAPE= MAPE(Y_test,LR_Test_predict)
print("MAPE: ",LR_MAPE)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
n_samples = 300
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 
import operator
from matplotlib import pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
data = pd.read_csv('../input/wages/wages.csv')
data = data.drop(data.columns[[1, 3, 5]], axis=1)
data.drop(data.index[100:], inplace=True)
data.loc[data.sex != 'female', 'female'] = 0
data.loc[data.sex == 'female', 'female'] = 1
data['sex'] = data['female']
data = data.drop(data.columns[[3]], axis=1)
print(data)



n = len(data)
mm = 3
m = 3
k = 3
MAX_ITER = 100


def printArr(arr):
    for i in range(n):
        for j in range(mm):
            print(arr[i][j], '  ', end = ' ')  ### Функция вывода матрицы
        print()
        
def subtract_vectors(v, w):
    return [vi - wi for vi, wi in zip(v, w)]  ## Функция вычитания векторов

def initialize():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]    ## Генерируется матрица u(i, j)
        summation = 1/sum(random_num_list)
        temp_list = [summation*x for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat



def calculateClusterCenter(membership_mat):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]                ####### Функция считает c
        denominator = sum(xraised) 
        temp_num = list()
        for i in range(n):
            data_point = list(data.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, list(zip(*temp_num)))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers



def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(data.iloc[i])
        distances = [np.linalg.norm(subtract_vectors(x, cluster_centers[j])) for j in range(k)] ## Обновили u
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat


def getClusters(membership_mat):
    cluster_labels = list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i])) ## Выбирает кластер для наблюдения
        cluster_labels.append(idx)
    return cluster_labels


def fuzzyCMeansClustering():
    # Membership Matrix
    membership_mat = initialize()
    curr = 0
    while curr <= MAX_ITER:
        cluster_centers = calculateClusterCenter(membership_mat)
        membership_mat2 = updateMembershipValue(membership_mat, cluster_centers)    # Сравнивает по эпсил. 
        cluster_labels = getClusters(membership_mat)
        curr += 1
    return membership_mat, membership_mat2, cluster_labels, cluster_centers

membership_mat,membership_mat2, fcm_labels, fcm_centers = fuzzyCMeansClustering()

fcm_labels = np.array(fcm_labels)
fcm_centers = np.array(fcm_centers)                       # Массивы в NumPy для рисунков
membership_mat2 = np.array(membership_mat2)
membership_mat = np.array(membership_mat)



        




import numpy as np
from sklearn.model_selection import train_test_split 
import pandas as pd
data = pd.read_csv('../input/wages/wages.csv')
data = data.drop(data.columns[[2, 3]], axis=1)
data.drop(data.index[300:], inplace=True)
data = data.drop(['ed'],axis=1) 
arr = data.to_numpy()

#Separating the dependent and independent data variables into two data frames.


for i in range(100):
    membership_mat[i][0]=(membership_mat[i][1]*membership_mat[i][2])/(membership_mat[i][1]+membership_mat[i][2])
for i in range(100):
    membership_mat[i][1]=(membership_mat[i][1]*membership_mat[i][2])/(membership_mat[i][1]+membership_mat[i][2])*arr[i][1]
for i in range(100):
    membership_mat[i][2]=(membership_mat[i][1]*membership_mat[i][2])/(membership_mat[i][1]+membership_mat[i][2])*arr[i][2]


df = pd.DataFrame(membership_mat, columns = ['earn','sex','ed'])
X = df 
Y = df['earn']
    
    
# Splitting the dataset into 80% training data and 20% testing data.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.20, random_state=0)
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

from sklearn.linear_model import LinearRegression
linear_model = LinearRegression().fit(X_train , Y_train)
 
#Predictions on Testing data
LR_Test_predict = linear_model.predict(X_test) 

 
# Using MAPE error metrics to check for the error rate and accuracy level
LR_MAPE= MAPE(Y_test,LR_Test_predict)
print("MAPE: ",LR_MAPE)
x_values = [0.0, 0.2, 0.4, 0.6, 0.8, 1]
f, axes = plt.subplots(1, 2, figsize=(11,5))
axes[0].scatter(membership_mat[:,0], membership_mat[:,1], alpha=.1)
axes[1].scatter(membership_mat[:,0], membership_mat[:,1], c=fcm_labels, alpha=.1)
xticks(x_values)
axes[1].scatter(fcm_centers[:,0], fcm_centers[:,1], marker="+", s=500, c='w')

plt.show()


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
n_samples = 300
import random
from sklearn.model_selection import train_test_split 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 
import operator
from matplotlib import pyplot as plt
imoex = pd.read_csv('../input/imoexnew/IMOEX_2.csv', sep=';')
imoex = imoex.drop(imoex.columns[[0,1,2,3,4,6,7,8,9,10,11,12,13,14]], axis=1)
arr = imoex.to_numpy()

g=6

for i in range (100 - g):
    summa = 0
    for j in range (i, i+g):
        summa += arr[j][0]
    arr[i][1] = summa / 6

for i in range (100 - g):
    arr[i][2] = ((arr[i][0] - arr[i][1]) / arr[i][1]) * 100
    
df = pd.DataFrame(arr, columns = ['close','sk','ot'])
df = df.drop(df[df["sk"] > 5000].index) 

X = df.drop(['close'],axis=1) 
Y = df['close']

 
# Splitting the dataset into 80% training data and 20% testing data.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.20, random_state=0)
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

from sklearn.linear_model import LinearRegression
linear_model = LinearRegression().fit(X_train , Y_train)

print('intercept:', linear_model.intercept_)
print('sk ot:', linear_model.coef_)
 
#Predictions on Testing data
LR_Test_predict = linear_model.predict(X_test) 

 
# Using MAPE error metrics to check for the error rate and accuracy level
LR_MAPE= MAPE(Y_test,LR_Test_predict)
print("MAPE: ",LR_MAPE)


n = len(df)
mm = 2
m = 2
k = 2
MAX_ITER = 100


def subtract_vectors(v, w):
    return [vi - wi for vi, wi in zip(v, w)]  ## Функция вычитания векторов

def initialize():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]    ## Генерируется матрица u(i, j)
        summation = 1/sum(random_num_list)
        temp_list = [summation*x for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat



def calculateClusterCenter(membership_mat):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]                ####### Функция считает c
        denominator = sum(xraised) 
        temp_num = list()
        for i in range(n):
            data_point = list(df.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, list(zip(*temp_num)))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers



def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(df.iloc[i])
        distances = [np.linalg.norm(subtract_vectors(x, cluster_centers[j])) for j in range(k)] ## Обновили u
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat


def getClusters(membership_mat):
    cluster_labels = list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i])) ## Выбирает кластер для наблюдения
        cluster_labels.append(idx)
    return cluster_labels


def fuzzyCMeansClustering():
    # Membership Matrix
    membership_mat = initialize()
    curr = 0
    while curr <= MAX_ITER:
        cluster_centers = calculateClusterCenter(membership_mat)
        membership_mat2 = updateMembershipValue(membership_mat, cluster_centers)    # Сравнивает по эпсил. 
        cluster_labels = getClusters(membership_mat)
        
        curr += 1
    
    return membership_mat, membership_mat2, cluster_labels, cluster_centers

membership_mat,membership_mat2, fcm_labels, fcm_centers = fuzzyCMeansClustering()

fcm_labels = np.array(fcm_labels)
fcm_centers = np.array(fcm_centers)                       # Массивы в NumPy для рисунков
membership_mat2 = np.array(membership_mat2)
membership_mat = np.array(membership_mat)
print(fcm_labels)
print('/////////////////')
print(membership_mat2)
print('/////////////////')


ar=np.zeros([94,2])

for j in range(2):
    for i in range (94):
        if (membership_mat2[i][j] > 1/k):
            ar[i][0] = membership_mat2[i][j]
        else:
            ar[i][1] = membership_mat2[i][j]
            
print(ar, 'cluster')
print('/////////////////')

ar2=np.zeros([94,1])

for i in range(94):
    ar2[i][0]=(ar[i][0]*ar[i][1])/(ar[i][0]*ar[i][1]) * 2
    
print(ar2)
            
        

x_values = [0.0, 0.2, 0.4, 0.6, 0.8, 1]
f, axes = plt.subplots(1, 2, figsize=(11,5))
axes[0].scatter(membership_mat[:,0], membership_mat[:,1], alpha=.1)
axes[1].scatter(membership_mat[:,0], membership_mat[:,1], c=fcm_labels, alpha=.1)
xticks(x_values)
axes[1].scatter(fcm_centers[:,0], fcm_centers[:,1], marker="+", s=500, c='w')

plt.show()
 

    
    